In [1]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import cv2
from lib.utils.augmentations import letterbox_for_img
import torchvision.transforms as transforms

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_img(path):
    img0 = cv2.imread(path, cv2.IMREAD_COLOR |
                      cv2.IMREAD_IGNORE_ORIENTATION)  # BGR
    h0, w0 = img0.shape[:2]

    img, ratio, pad = letterbox_for_img(img0.copy(), new_shape=640, auto=True)
    h, w = img.shape[:2]
    shapes = (h0, w0), ((h / h0, w / w0), pad)
    img = np.ascontiguousarray(img)
    return img, img0, shapes

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

In [20]:
f = open("/home/YOLOP/tr_export/bb.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

# need to set input and output precisions to FP16 to fully enable it
output = np.empty([1, 1000], dtype=np.float16)

# allocate device memory     5000000
d_input = cuda.mem_alloc(5000000)
d_output = cuda.mem_alloc(5000000)
# (1474560,4669440)
bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

img, img_det, shapes = load_img(
    '/home/YOLOP/inference/images/adb4871d-4d063244.jpg')
img = transform(img)
if img.ndimension() == 3:
            img = img.unsqueeze(0)
img = np.half(img)
print(img.shape)

input_idx = engine['img']
output_idx = engine['out_bb0']


(1, 3, 384, 640)


In [17]:
def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod(d_input, batch)
    # execute model
    context.execute(bindings, None)
    # transfer predictions back
    cuda.memcpy_dtoh(output, d_output)
    # syncronize threads
    stream.synchronize()
    
    return output

In [18]:
print("Warming up...")

pred = predict(img)

print("Done warming up!")

Warming up...


TypeError: execute(): incompatible function arguments. The following argument types are supported:
    1. (self: tensorrt.tensorrt.IExecutionContext, batch_size: int = 1, bindings: List[int]) -> bool

Invoked with: <tensorrt.tensorrt.IExecutionContext object at 0x7f74f01737f0>, [140142289354752, 140142337589248], None